# Flower Identifier

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm as notebook_tqdm

In [19]:
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

In [20]:
train_dataset, test_dataset, val_dataset = dataset['train'], dataset['test'], dataset['validation']

In [21]:
IMG_SIZE = 224

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize the image to 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the pixel values to [0, 1]
    return image, label

In [22]:
train_dataset = train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [23]:
# Load model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [24]:
# Freeze the base model
base_model.trainable = False

# Build the final model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(102, activation='softmax')
])

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 926ms/step - accuracy: 0.0505 - loss: 4.7364 - val_accuracy: 0.3637 - val_loss: 3.1148
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 913ms/step - accuracy: 0.6318 - loss: 2.2974 - val_accuracy: 0.6412 - val_loss: 2.0469
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 878ms/step - accuracy: 0.8922 - loss: 1.2155 - val_accuracy: 0.7108 - val_loss: 1.5793
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 950ms/step - accuracy: 0.9532 - loss: 0.7439 - val_accuracy: 0.7461 - val_loss: 1.3396
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 941ms/step - accuracy: 0.9765 - loss: 0.4965 - val_accuracy: 0.7598 - val_loss: 1.2001
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 909ms/step - accuracy: 0.9896 - loss: 0.3531 - val_accuracy: 0.7676 - val_loss: 1.1097
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 887ms/step - accuracy: 0.9979 - loss: 0.2632 - val_accuracy: 0.7716 - val_loss: 1.0464
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 1.0000 - loss: 0.2037 - val_accurac

In [27]:
# Unfreeze base model
base_model.trainable = True

# Freeze all layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Re-compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [28]:
history_finetune = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9713 - loss: 0.2823 - val_accuracy: 0.7853 - val_loss: 0.9245
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 945ms/step - accuracy: 0.9776 - loss: 0.2391 - val_accuracy: 0.7824 - val_loss: 0.9174
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 901ms/step - accuracy: 0.9848 - loss: 0.2089 - val_accuracy: 0.7843 - val_loss: 0.9118
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 908ms/step - accuracy: 0.9891 - loss: 0.1838 - val_accuracy: 0.7833 - val_loss: 0.9072
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 911ms/step - accuracy: 0.9898 - loss: 0.1628 - val_accuracy: 0.7833 - val_loss: 0.9033
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 911ms/step - accuracy: 0.9919 - loss: 0.1453 - val_accuracy: 0.7843 - val_loss: 0.8997
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 911ms/step - accuracy: 0.9948 - loss: 0.1305 - val_accuracy: 0.7833 - val_loss: 0.8962
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 922ms/step - accuracy: 0.9970 - loss: 0.1179 - val_accurac

In [29]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

193/193 ━━━━━━━━━━━━━━━━━━━━ 85s 440ms/step - accuracy: 0.7583 - loss: 1.0023
Test accuracy : 0.7576841711997986
